In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import sys
import os
import collections
from tqdm import tqdm
HERE = %pwd
sys.path.append(os.path.dirname(HERE))

1. Download from https://www.kaggle.com/datasets/yelp-dataset/yelp-dataset
1. Unzip the file.
1. Place the dataset into `dir_save`

In [2]:
# data directory that save raw data
dir_save = "../data/raw_data/yelp"

import json
def _load(data_name):
    data_file = open(f"{dir_save}/yelp_academic_dataset_{data_name}.json")

    data = [json.loads(line) for line in tqdm(data_file)]
    df = pd.DataFrame(data)
    data_file.close()
    return df

In [3]:
# item master
df_items = _load("business").rename(columns={"business_id" : "itemID"}).set_index("itemID")
df_items = df_items.drop(columns=["address", "postal_code", "latitude", "longitude", "hours"])
df_items.head()

150346it [00:03, 41232.66it/s]


,name,city,state,stars,review_count,is_open,attributes,categories
itemID,,,,,,,,
Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ",Santa Barbara,CA,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop..."
mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,Affton,MO,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma..."
tUFrWirKiKi_TAnsVWINQQ,Target,Tucson,AZ,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G..."
MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,Philadelphia,PA,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,Green Lane,PA,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food"


In [4]:
# transaction 
df_review = _load("review").rename(columns={"business_id" : "itemID", "user_id" : "userID"})
df_review.head()

6990280it [01:05, 106499.36it/s]


,review_id,userID,itemID,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [5]:
# directory to save preprocessed data to use experiments
version_prep = "20250403_prep"
dir_save_data = f"../data/preprocessed_data/{version_prep}/Yelp"
os.makedirs(dir_save_data, exist_ok=True)

# groupby
s = df_review["userID"].value_counts()
gb = df_review.groupby("userID")
l_item_column_to_select = ["city", "state", "is_open"]

# heavy or light
du = dict()
du["light"] = s[(s>5+1)*(s<=10+1)].index.values
du["heavy"] = s[(s>30+1)*(s<=50+1)].index.values

di = dict()
for type_user, users_ in du.items():
    idx = 0
    dr = dict()
    d_ = dict()
    for user in tqdm(users_):
        df_r = gb.get_group(user).sort_values(by="date", ascending=True)
        # items except for the lastest item
        items_train = df_r["itemID"].values[:-1]
    
        # latest item
        s_last = df_r.iloc[-1]
        if s_last["stars"] > 3:
            item_test = s_last["itemID"]
            try:
                # select items that have the same l_item_column_to_select
                s_ = df_items.loc[item_test]
                df_ = pd.concat([df_items[a] == s_[a] for a in l_item_column_to_select], axis=1)
                df_ = df_items[df_.T.sum() == len(l_item_column_to_select)]
                df_ = df_.drop(columns=l_item_column_to_select)
    
                # candidate
                items_others = df_.sample(n=9, replace=False).index.values
                items_candi = sorted(set(items_others).union(set([item_test])))
                
                d_[user] = {
                    "id_train" : items_train, 
                    "id_candi" : items_candi
                }
                df_ = df_r[['userID', 'itemID', 'stars', 'useful', 'funny', 'cool', 'text']]
                df_ = df_.rename(columns={"stars" : "rating", "text" : "review"})
                dr[user] = df_
                idx += 1
            except:
                pass
        
        ## +5 is for supplementary
        if idx == 200+5:
            break

    # items for experiments
    di.update(d_)
    
    # save
    import pickle
    with open(f"{dir_save_data}/ids_{type_user}.pickle", 'wb') as f:
        pickle.dump(d_, f)
        
    df_r = pd.concat(dr.values())
    df_r.to_csv(f"{dir_save_data}/records_{type_user}.csv")

# items that appeared for experiments
items_train = np.unique(np.concatenate([d["id_train"] for d in di.values()]))
items_candi = np.unique(np.concatenate([d["id_candi"] for d in di.values()]))
items = sorted(set(items_train).union(set(items_candi)))

df_items_valid = df_items.loc[items]
df_items_valid.to_csv(f"{dir_save_data}/items_slim.csv")
print("finished")

  2%|█▌                                                              | 281/11534 [00:23<15:55, 11.78it/s]


finished


In [6]:
len(df_items_valid)

12902

In [7]:
df_r

,userID,itemID,rating,useful,funny,cool,review
1791813,IrxksmWRD9rRivs7JE6IKg,2pDXWt8KnncH7FEHL25NvA,5.0,2,1,0,HIGHLY RECOMMENDED: We called about 20minutes ...
6718349,IrxksmWRD9rRivs7JE6IKg,uxtkqKLsVeXZz-5AQ1uI3A,4.0,1,0,1,I haven't any run ins with the owner.. But I w...
1863467,IrxksmWRD9rRivs7JE6IKg,ChMvzo7WYSZzxlwGMS7CFA,1.0,3,0,0,Wouldn't come back. I was very surprised at th...
1844859,IrxksmWRD9rRivs7JE6IKg,ZygSmxsAa1qXp5oiacIbew,4.0,1,0,0,HIGHLY RECOMMENDED : The food is undeniably gr...
1841283,IrxksmWRD9rRivs7JE6IKg,jukJ94bXlgu4KU4x8xANpg,3.0,1,0,0,NOT AN AWARD WINNING EXPERIENCE: Ordered 4 pie...
...,...,...,...,...,...,...,...
6512716,GrexTlTuv9UfCQbGA_d_wQ,eDLnzdEsJTYmqVdRheOyrw,4.0,1,0,0,One of the better Dunkin Donuts experiences. T...
4265077,GrexTlTuv9UfCQbGA_d_wQ,FwvK-Xyi11yDeYmqmL6uSw,2.0,1,0,1,"The Starbucks, on the surface, is pretty solid..."
3694037,GrexTlTuv9UfCQbGA_d_wQ,smJmkoRsklJj5ZoxK--wpg,5.0,2,0,0,Anna's might be the best sandwich shop in Delc...
1678953,GrexTlTuv9UfCQbGA_d_wQ,Rt4xYQBWC8i2xqLp9dP7XQ,4.0,0,0,0,I lived in Ridley for two years before I learn...


In [8]:
df_items_valid

,name,city,state,stars,review_count,is_open,attributes,categories
itemID,,,,,,,,
--epgcb7xHGuJ-4PUeSLAw,Manhattan Bagel,Willow Grove,PA,3.0,34,1,"{'BikeParking': 'True', 'RestaurantsPriceRange...","Restaurants, Food, Bagels, Sandwiches, Breakfa..."
-02xFuruu85XmDn2xiynJw,Family Vision Center,Tucson,AZ,4.5,109,1,"{'ByAppointmentOnly': 'True', 'BusinessParking...","Shopping, Ophthalmologists, Optometrists, Doct..."
-09Oc2D14vRnmirPh0vlXw,Cracker Barrel Old Country Store,Mount Holly,NJ,3.0,135,1,"{'WiFi': 'u'no'', 'RestaurantsAttire': 'u'casu...","Diners, Shopping, Caterers, Restaurants, Comfo..."
-0TffRSXXIlBYVbb5AwfTg,IndeBlue Modern Indian Food & Spirits,Philadelphia,PA,4.5,1097,1,"{'RestaurantsReservations': 'True', 'NoiseLeve...","Cocktail Bars, Food Delivery Services, Nightli..."
-0__F9fnKt8uioCKztF5Ww,Piscobar,New Orleans,LA,4.0,66,1,"{'RestaurantsAttire': ''casual'', 'WheelchairA...","Cafes, Nightlife, Cocktail Bars, Peruvian, Res..."
...,...,...,...,...,...,...,...,...
zy_g2wKTNIB7EQdG73_Xaw,Einstein Medical Center Philadelphia,Philadelphia,PA,2.0,44,1,"{'ByAppointmentOnly': 'False', 'BusinessAccept...","Hospitals, Health & Medical, Medical Centers, ..."
zymvTQ12tRy3bzmP4tPgOg,Salt + Smoke,Saint Charles,MO,4.0,368,1,"{'Caters': 'True', 'RestaurantsTakeOut': 'True...","Beer Bar, Sandwiches, Bars, Barbeque, Restaura..."
zzG-EObaHskhFLy6suavpA,G&H Tile Masters,Skippack,PA,5.0,6,1,{'BusinessAcceptsCreditCards': 'True'},"Flooring, Contractors, Home Services"
